# Import things

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import random
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
random.seed(42)

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Load data

In [2]:
red_wine = pd.read_csv("./resources/resources/winequality-red.csv", sep=';')

# V.1 Exploring the green reds

In [3]:
red_wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


## a)

In [4]:
def plot_scatter_matrix(wine_data, good_threshhold, bad_threshold, save_plot=False):
    """Visualizes the data
    """
    length = len(wine_data.columns)
    fig, ax = plt.subplots(length, length, figsize=(50, 50))
    bigger = wine_data.where(wine_data['quality'] > good_threshhold)
    smaller = wine_data.where(wine_data['quality'] < bad_threshold)
    
    for i in range(length):
        for j in range(length):
            if i == j:
                ax[(i, j)].text(0.5, 0.5, '\n'.join(wine_data.columns[i].split()), fontsize=35, ha='center', va='center')
            else:
                ax[(j, i)].plot(bigger.iloc[:, i], bigger.iloc[:, j], 'co')
                ax[(j, i)].plot(smaller.iloc[:, i], smaller.iloc[:, j], 'mo')
    if save_plot:
        plt.savefig('scatter_matrix.png')
# plot_scatter_matrix(red_wine, 6, 5)

In [5]:
# plot_scatter_matrix(red_wine, 7, 4)

## b)

I think it can be pH, sulphates, alcohol and total sulfur dioxide. Cause there
different
groups of wine(based on quality) r better splitted.

# V.2 Learning to perceptron

In [6]:
data = red_wine[(red_wine['quality'] >= 8) | (red_wine['quality'] <= 3)].reset_index()
y = [1 if i == True else 0 for i in data['quality'] >= 8]

In [7]:
def display(ret, dot=100, detail=10000, final=False):
    """Displays text info about training process"""
    if final:
        print()
        print('DONE')
        print('cycle = ', ret[-1][0], ', errors = ', ret[-1][1], sep='')
        print('final weights = ', ret[-1][2], ', bias = ', ret[-1][3])
    if ret[-1][0] == 0:
        return
    if ret[-1][0] % dot == 0:
        print('.', end='')
    if ret[-1][0] % detail == 0:
        print()
        print('cycle = ', ret[-1][0], ', errors = ', ret[-1][1], sep='')
#         print(ret[-1])
#         print(ret[-dot])
        print('weights change = ', ret[-1][2][0] - ret[-dot - 1][2][0], ret[-1][2][1] - ret[-dot - 1][2][1])
        print('bias change = ', ret[-1][3] - ret[-dot - 1][3])

## a), b)

In [8]:
class perceptron:
    """A simple perceptron, with randomly initialized weights and bias"""

    def __init__(self, amount):
        """Random data initialization"""
        self.amount = amount
        self.weights = [random.random() for n in range(amount)]
        self.bias = random.random()

    def activation(self, value):
        """Activation function"""
        return 1 if value > 0 else 0

    def predict(self, data, alpha = None, y = None):
        """Predicts answer and updates weights to fit the data item better
        Returns predicted value or whether he guessed right
        """
        rez = self.activation(sum([data[i] * self.weights[i] for i in range(self.amount)]) + self.bias)
        if y == None:
            return rez
        if rez != y:
            self.weights = [self.weights[i] + alpha * data[i] * (y - rez) for i in range(self.amount)]
            self.bias = self.bias + alpha * (y - rez)
        return rez != y

    def train(self, data, y, epochs = 0, alpha = 0.001):
        """Trains perceptron epoch times, and logs results"""
        ret = []
        errors = 1
        if epochs == 0:
            i = 0
            while (errors):
                errors = 0
                for item in data.itertuples():
                    errors += self.predict(item[1:], alpha, y[item[0]])
                ret.append([i, errors, self.weights, self.bias])
                i += 1
                display(ret)
            display(ret, final=True)
        else:
            for i in range(epochs):
                errors = 0
                for item in data.itertuples():
                    errors += self.predict(item[1:], alpha, y[item[0]])
                ret.append([i, errors, self.weights, self.bias])
                display(ret)
                if errors == 0:
                    break
            display(ret, final=True)
        return ret

In [9]:
model = perceptron(2)
logs = model.train(data[['pH', 'alcohol']], y, 0, 0.01)

....................................................................................................
cycle = 10000, errors = 2
weights change =  -0.03679999999995598 -0.05800000000000005
bias change =  0.049999999999998934
...............................
DONE
cycle = 13154, errors = 0
final weights =  [-7.259373201536793, 1.731510755222712] , bias =  6.135029318369033


## c)

In [10]:
def plot_performance(performance, wine_data, good_threshold, bad_threshold, epoch=-1, save_plot=False):
    """Plot the performance of my perceptron"""
    fig, ax = plt.subplots(1, 2)#, figsize=(10, 10))
    bigger = wine_data.where(wine_data['quality'] > good_threshold).reset_index()
    smaller = wine_data.where(wine_data['quality'] < bad_threshold).reset_index()
    
    ax[0].plot([i[1] for i in performance], 'g')
    ax[0].set_title('Errors as a function of epoch')
    ax[0].set_ylabel('Classification errors')
    ax[0].set_xlabel('Epoch')
    ax[1].plot(bigger['alcohol'], bigger['pH'], 'mo', label = 'good wines (> ' + str(good_threshold) + ' score)')
    ax[1].plot(smaller['alcohol'], smaller['pH'], 'co', label = 'bad wines (< ' + str(bad_threshold) + ' score)')
    ax[1].set_title('Decision boundary on epoch:' + str(len(performance) if epoch == -1 else epoch))
    ax[1].set_xlabel('Alcohol')
    ax[1].set_ylabel('pH')
    
    val = {}
    val['alcohol'] = [min(wine_data['alcohol']), max(wine_data['alcohol'])] 
    val['pH'] = [
            - (performance[epoch][2][1] * val['alcohol'][0] + performance[epoch][3]) / performance[epoch][2][0],
            - (performance[epoch][2][1] * val['alcohol'][1] + performance[epoch][3]) / performance[epoch][2][0]
        ]
    ax[1].plot(val['alcohol'], val['pH'], 'g--')
    ax[1].fill_between(val['alcohol'], val['pH'], val['pH'][0], facecolor='#b2ebf2')
    ax[1].fill_between(val['alcohol'], val['pH'], val['pH'][1], facecolor='#f8bbd0')
    ax[1].set_xlim(min(wine_data['alcohol']), max(wine_data['alcohol']))
    ax[1].set_ylim(min(wine_data['pH']), max(wine_data['pH']))
    plt.tight_layout()
    plt.legend(loc='upper center', bbox_to_anchor=(1.6, 1))
    plt.show()

In [11]:
interactive(plot_performance, performance=fixed(logs), wine_data=fixed(red_wine), good_threshold=(0, 10),\
            bad_threshold=(0, 10), epoch=(0, len(logs) - 1))

interactive(children=(IntSlider(value=5, description='good_threshold', max=10), IntSlider(value=5, description…

## d) Feature scaling

In [12]:
normalized_red_wine = red_wine.copy()
normalized_red_wine['pH'] = (red_wine['pH'] - min(red_wine['pH'])) / (max(red_wine['pH']) - min(red_wine['pH']))
normalized_red_wine['alcohol'] = (red_wine['alcohol'] - min(red_wine['alcohol'])) / (max(red_wine['alcohol']) - min(red_wine['alcohol']))
normalized_data = normalized_red_wine[(red_wine['quality'] >= 8) | (red_wine['quality'] <= 3)].reset_index()
# data.describe()

In [13]:
model_normalized = perceptron(2)
logs_normalized = model_normalized.train(normalized_data[['pH', 'alcohol']], y, 0, 0.01)

.
DONE
cycle = 119, errors = 0
final weights =  [-0.33482075791416915, 0.42054813724093704] , bias =  0.02669948742291084


In [14]:
interactive(plot_performance, performance=fixed(logs_normalized), wine_data=fixed(normalized_red_wine), good_threshold=(0, 10),\
            bad_threshold=(0, 10), epoch=(0, len(logs_normalized) - 1))

interactive(children=(IntSlider(value=5, description='good_threshold', max=10), IntSlider(value=5, description…

# V.3 My fair ADALINE

## a)

Using stochastic learning algorithm instead of batch. Not adding polynomial features and not using more complicated algorhithms. No parallelism. Using only labels in learning, not values.

## b), c)

In [15]:
class adaline:
    """A simple adaline, with randomly initialized weights and bias"""

    def __init__(self, amount):
        """Random data initialization"""
        self.amount = amount
        self.weights = [random.random() for n in range(amount)]
        self.bias = random.random()

        
    def quantizer(self, value):
        """Quantizer function"""
        return 1 if value > 5 else 0

    
    def predict(self, data, y=False):
        """Predicts answer and updates weights to fit the data item better
        Returns predicted value or whether he guessed right
        """
        rez = sum([data[i] * self.weights[i] for i in range(self.amount)]) + self.bias
#         print('potaot', self.quantizer(rez))
        if y == None:
            return self.quantizer(rez)
        else:
#             print(rez, y)
            return [x * (y - rez) for x in [*data, 1]]

        
    def train(self, data, y, epochs = 0, alpha = 0.001):
        """Trains perceptron epoch times, and logs results"""
        ret = []
        if epochs == 0:
            i = 0
            while (True):
                errors = 0
                delta = [0] * (self.amount + 1)
                for item in data.itertuples():
                    rez = self.predict(item[1:], y[item[0]])
                    delta = [delta[i] + rez[i] for i in range(self.amount + 1)]
                    if self.predict(item) != (1 if y[item[0]] >= 7 else 0):
                        errors += 1
                self.weights = [self.weights[i] + alpha * delta[i] / len(data) for i in range(self.amount)]
                self.bias += alpha * delta[-1] / len(data)
                ret.append([i, errors, self.weights, self.bias])
                i += 1
                display(ret)
                if errors == 0:
                    break
            display(ret, final=True)
        else:
            for i in range(epochs):
                errors = 0
                delta = [0] * (self.amount + 1)
                for item in data.itertuples():
                    rez = self.predict(item[1:], y[item[0]])
                    delta = [delta[i] + rez[i] for i in range(self.amount + 1)]
                    if self.predict(item) != (1 if y[item[0]] >= 7 else 0):
                        errors += 1
                self.weights = [self.weights[i] + alpha * delta[i] / len(data) for i in range(self.amount)]
                self.bias += alpha * delta[-1] / len(data)
                ret.append([i, errors, self.weights, self.bias])
                display(ret)
                if not any(delta):
                    break
            display(ret, final=True)
        return ret

In [16]:
data_4_7 = red_wine[(red_wine['quality'] >= 7) | (red_wine['quality'] <= 4)].reset_index()
y_4_7 = data_4_7['quality']

In [17]:
model = adaline(2)
adaline_logs = model.train(data_4_7[['pH', 'alcohol']], y_4_7.tolist(), 10000, 0.001)

...................................................................................................
DONE
cycle = 9999, errors = 280
final weights =  [-0.292573386138416, 0.6249294138274334] , bias =  0.30646077848770475


In [18]:
for l in adaline_logs[:20]:
    print(l)

[0, 280, [0.8987249790691545, 0.11005456314616571], 0.4239242383089659]
[1, 280, [0.9043320221687786, 0.12996280139926616], 0.42564351211508217]
[2, 280, [0.9091312516939356, 0.14710979311002015], 0.4271190347735665]
[3, 280, [0.9132350587385878, 0.16187969946686223], 0.42838471925724925]
[4, 280, [0.9167401978350704, 0.1746032347190518], 0.4294697603511621]
[5, 280, [0.9197299624116468, 0.18556510205358762], 0.43039929107672775]
[6, 280, [0.9222760575866208, 0.1950103949457992], 0.431194947790056]
[7, 280, [0.924440212407463, 0.203150107913473], 0.43187535666018306]
[8, 280, [0.9262755677850115, 0.21016588057993868], 0.43245655246537545]
[9, 280, [0.9278278713294703, 0.21621408171303177], 0.4329523391238375]
[10, 280, [0.9291365059532509, 0.22142932506667196], 0.43337460006510214]
[11, 280, [0.9302353753680676, 0.22592749607627816], 0.4337335654205864]
[12, 280, [0.9311536663860643, 0.22980835746113504], 0.4340380420409036]
[13, 280, [0.9319165051647782, 0.23315779231884423], 0.434295

In [19]:
interactive(plot_performance, performance=fixed(adaline_logs), wine_data=fixed(data_4_7), good_threshold=(0, 10),\
            bad_threshold=(0, 10), epoch=(0, len(adaline_logs) - 1))

interactive(children=(IntSlider(value=5, description='good_threshold', max=10), IntSlider(value=5, description…